In [1]:
pip install Flask pandas numpy scikit-learn nltk



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
data = pd.read_csv('WELFake_Dataset.csv')
data.tail()

,Unnamed: 0,title,text,label
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0
72133,72133,Goldman Sachs Endorses Hillary Clinton For Pre...,Goldman Sachs Endorses Hillary Clinton For Pre...,1


In [3]:
data.isnull().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [4]:
data = data.rename(columns={'Unnamed: 0': 'id'})


In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Download NLTK stopwords data
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ujjwalsamanta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ujjwalsamanta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Assuming 'data' is your DataFrame
# If 'text' column contains non-string elements, you can convert them to string
data['text'] = data['text'].astype(str)

# Function to remove HTML tags
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Function to remove URLs
def remove_urls(text):
    return re.sub(r'http\S+', '', text)

# Function to preprocess text
def preprocess_text(text):
    # Remove HTML tags
    text = remove_html_tags(text)
    
    # Remove URLs
    text = remove_urls(text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove punctuation
    tokens = [word for word in tokens if word.isalpha()]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]

    # Remove numbers
    tokens = [word for word in tokens if not word.isdigit()]

    # Lowercasing
    tokens = [word.lower() for word in tokens]

    # Remove extra whitespaces
    text = ' '.join(tokens)

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

# Apply preprocessing to the 'text' column
data['preprocessed_text'] = data['text'].apply(preprocess_text)

# Display the result
print(data[['text', 'preprocessed_text']])


                                                    text  \
0      No comment is expected from Barack Obama Membe...   
1         Did they post their votes for Hillary already?   
2       Now, most of the demonstrators gathered last ...   
3      A dozen politically active pastors came here f...   
4      The RS-28 Sarmat missile, dubbed Satan 2, will...   
...                                                  ...   
72129  WASHINGTON (Reuters) - Hackers believed to be ...   
72130  You know, because in fantasyland Republicans n...   
72131  Migrants Refuse To Leave Train At Refugee Camp...   
72132  MEXICO CITY (Reuters) - Donald Trump’s combati...   
72133  Goldman Sachs Endorses Hillary Clinton For Pre...   

                                       preprocessed_text  
0      comment expect barack obama member fukyoflag b...  
1                              post vote hillari alreadi  
2      demonstr gather last night exercis constitut p...  
3      dozen polit activ pastor came privat

In [22]:
# Tokenize the preprocessed text
data['tokenized_text'] = data['preprocessed_text'].apply(lambda x: x.split())


In [23]:
from gensim.models import Word2Vec

# Training the Word2Vec model on the tokenized text
tokenized_text = data['tokenized_text'].tolist()
word2vec_model = Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, workers=4)


In [24]:
import numpy as np

def document_vector(word2vec_model, doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if not doc:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word2vec_model.wv[doc], axis=0)


In [25]:
# Vectorize the dataset
X = np.array([document_vector(word2vec_model, doc) for doc in tokenized_text])

# Assuming your labels are in the 'label' column
y = data['label'].values


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [35]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

50493
21641
50493
21641


In [30]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the classifier
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)


RandomForestClassifier()

In [31]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Generate a classification report
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.999306871216672
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10613
           1       1.00      1.00      1.00     11028

    accuracy                           1.00     21641
   macro avg       1.00      1.00      1.00     21641
weighted avg       1.00      1.00      1.00     21641



In [36]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load dataset
data = pd.read_csv('WELFake_Dataset.csv')
data['text'] = data['text'].astype(str)

# Function to preprocess text
def preprocess_text(text):
    # Remove HTML tags and URLs
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+', '', text)
    
    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

# Apply preprocessing to the 'text' column
data['preprocessed_text'] = data['text'].apply(preprocess_text)

# Tokenize the preprocessed text for Word2Vec
data['tokenized_text'] = data['preprocessed_text'].apply(lambda x: x.split())

# Train Word2Vec model
word2vec_model = Word2Vec(data['tokenized_text'].tolist(), vector_size=100, window=5, min_count=1, workers=4)

# Function to create document vectors
def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if not doc:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word2vec_model.wv[doc], axis=0)

# Vectorize the dataset
X = np.array([document_vector(word2vec_model, doc) for doc in data['tokenized_text']])
y = data['label'].values

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train RandomForestClassifier with some basic hyperparameters
classifier = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
classifier.fit(X_train, y_train)

# Evaluate the classifier using cross-validation on the training set
cv_scores = cross_val_score(classifier, X_train, y_train, cv=5)
print(f"CV Scores: {cv_scores.mean()}")

# Evaluate on the test set
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))


CV Scores: 0.8973522221118081
              precision    recall  f1-score   support

           0       0.92      0.88      0.90     10613
           1       0.89      0.92      0.90     11028

    accuracy                           0.90     21641
   macro avg       0.90      0.90      0.90     21641
weighted avg       0.90      0.90      0.90     21641



In [37]:
pip install transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 12.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 9.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.4/426.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 36.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1
ERROR: pip's dependency resolver does

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


### LSTM

In [ ]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense


In [1]:
# Tokenization
texts = data['preprocessed_text'].tolist()
labels = data['label'].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Padding sequences
maxlen = 100  # Assuming maximum sequence length
sequences = pad_sequences(sequences, maxlen=maxlen)


NameError: name 'data' is not defined

In [ ]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)


In [ ]:
# Word2Vec Embedding
word2vec_model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4)
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]


In [ ]:
# LSTM Model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from joblib import dump

# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Save the trained model
model_path = 'LSTM_fake_news_detection_model.joblib'
dump(model, model_path)

print("Model saved successfully as", model_path)

In [ ]:
# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


In [ ]:
# Classification Report
y_pred = model.predict_classes(X_test)
print(classification_report(y_test, y_pred))
